In [1]:
import pyodbc
import pandas as pd
from datetime import datetime
import hashlib as hs

In [148]:
def conectar(config):
    cnxn = pyodbc.connect('DSN='+ config[5], SERVER=config[0] + ',' + config[1],UID=config[3], PWD=config[4])
    crsr = cnxn.cursor()
    row  = crsr.execute("SELECT * FROM "+config[6]).fetchone()
    #cnxn.close()
    if row =='':
        print('Conexão falhou')
    else:
        print('Conexão ok')
    return  cnxn

#passa o dtaframe e o nome das colunas em que deve ser aplicado o algoritmo de anonimização
def hashingData2 (dfOrigin, colName): #recebe o dataframe e a lista com nome de colunas que contém em seu título CPF, Nome ou RG
    df=dfOrigin
    for c in colName:
        nomecoluna_hs = c +'_hs'
        df[c] = df[c].astype(str)
        #Apply hashing function to the column
        df[nomecoluna_hs] = df[c].apply(lambda x: hs.sha256(x.encode()).hexdigest())
        df.drop(c,axis=1,inplace=True)
    return df

def hashingData (dfOrigin, colName): #recebe o dataframe e a lista com nome de colunas que contém em seu título CPF, Nome ou RG
    df=dfOrigin
    arrayNovoValor = []
    for c in colName:
        nomecoluna_hs = c +'_hs'
        df[c] = df[c].astype(str)
        #Apply hashing function to the column
        df[c].apply(lambda x: arrayNovoValor.append(hashing(x)))
        df[nomecoluna_hs] = arrayNovoValor
        df.drop(c,axis=1,inplace=True)
    return df


def hashing(valor):
    if valor == None:
        valorFinal = ''
    else:
        valorFinal = hs.sha256(valor.encode()).hexdigest()
    return valorFinal


In [45]:
#traz argumentos para acessar as informações
f=open("entrada.csv","r")
lines=f.readlines()
argumento = []
valor =[]
for x in lines:
    argumento.append(x.split(";")[0])
    valor.append(x.split(";")[1].split("\n")[0])               
f.close()   
server = valor[0]
serverport = valor[0] + ',' + valor[1]  #endereço vírgula porta
port =valor[1]
database = valor[2]
username = valor[3]
password = valor[4]
dsn = valor[5]
table_ = valor[6]

In [74]:
conexao = conectar(valor)
cursor = conexao.cursor()
nomeColunas =[]
for row in cursor.columns(table=table_[7:]):
    nomeColunas.append(row.column_name)
conexao.close()
len(nomeColunas)

113

In [77]:
#para adicionar uma nova coluna ao Select deve-se inserir o nome dela no arquivo txt
col=open("colunas.txt",'r')
lines=col.readlines()
colu = []
for x in lines:
    colu.append(x.split("'")[1])
col.close() 
colunasUsadas=colu

In [78]:
dfColunas = pd.DataFrame({'NomeColunaDisponível':nomeColunas}) #adiciona o nome de todas as colunas disponíveis em um df
dfColunas['ColunasUteis']= dfColunas.apply(lambda x: x['NomeColunaDisponível'] in colunasUsadas, axis=1) #adiciona "1" se coluna será utilizada e "0" se não for

In [ ]:
for x in dfColunas.loc[(dfColunas['ColunasUteis']==True)]['NomeColunaDisponível']:
    print(x)

In [144]:
#montagem da string sql que vai buscar apenas as colunas necessárias ao estudo
stringSelect =''
for x in dfColunas.loc[(dfColunas['ColunasUteis']==True)]['NomeColunaDisponível']:
    stringSelect = stringSelect + str(x) +', '
conexao = conectar(valor) #realiza coneção e imprime se obteve sucesso ou não
sql = 'SELECT '+ stringSelect[0:-2] + ' FROM ' + table_ #stringSelect[0:-2] #pega string sem a ultima vírgula
print('Hora de Início da consulta:', datetime.now())
dfBase = pd.read_sql(sql,conexao) #realização da pesquisa na base
conexao.close()
print('Hora de fim da consulta',datetime.now())

Conexão ok
Hora de Início da consulta: 2021-11-30 17:33:03.372507
Hora de fim da consulta 2021-11-30 17:42:39.394562


In [166]:
#Teste
#pesquisa = cursor.execute('SELECT '+ stringSelect[0:-2] + ' FROM RIC_DW.RIC_SBX_DETRAN_REQUERIMENTO').fetchone()

In [145]:
colunasAnonimizaveis = []
for namecol in colunasUsadas:
    if (namecol.find('CPF')!= -1 or 
    (namecol.find('NOME') != -1 and namecol.find('MUN') == -1 ) or
    namecol.find('RG_')!=-1 ):
        colunasAnonimizaveis.append(namecol)  
#colunasAnonimizaveis     

In [150]:
dfBase.head()

,NRO_REQ,OAT,NSU,COD_SITUACAO,DESCRICAO_SITUACAO,COD_SERV,DESCRICAO_SERV,DESCR_RECEB,DATA_POSTAGEM,COD_ESTADO,...,RG_CONDUTOR,CONDUTOR_IDADE,CONDUTOR_SEXO,CONDUTOR_DATA_NASCIMENTO,CONDUTOR_VALIDADE_CNH,NOME_MUNIC_CONDUTOR,REQUERENTE_IDADE,REQUERENTE_SEXO,REQUERENTE_DATA_NASCIMENTO,NOME_REQUERENTE_hs
0,202107888555,286470.0,E019706665,70,Em execução,6,Apresentação de Condutor,ECT,2021-07-07,2,...,,NaN,Não Informado,None,None,None,NaN,Não Informado,None,33006c87e302ef434d97f78fa7cdf3aafc8311c6784fcd...
1,202009328962,288010.0,E019481851,80,Executado,6,Apresentação de Condutor,ECT,2020-05-29,9,...,,NaN,Não Informado,None,None,None,NaN,Não Informado,None,b2cc09045ecbcbb002fed3e424448953de92dce9c9cf06...
2,202105046743,285610.0,E019997740,70,Em execução,6,Apresentação de Condutor,ECT,2021-05-04,2,...,,NaN,Não Informado,None,None,None,NaN,Não Informado,None,e96ccf773e8067a62f75ecd48c7f762ab2f60b289d9603...
3,202112608379,288010.0,E020195017,20,Em conferência,6,Apresentação de Condutor,Protocolo Original,None,1,...,,NaN,Não Informado,None,None,None,NaN,Não Informado,None,791d57579051ffb56bde0e9949e401ae6abf7e45a46c21...
4,202104922736,287850.0,TE00017125,70,Em execução,6,Apresentação de Condutor,ECT,2021-04-29,2,...,,NaN,Não Informado,None,None,None,NaN,Não Informado,None,6827ba4355dd626e70d5bb04dfb556c1df64a572d44dd3...


In [149]:
dfAnonimo = dfBase
dfAnonimo = hashingData(dfAnonimo,colunasAnonimizaveis) #anonimiza dados
dfAnonimo.head()

ValueError: Length of values (3785126) does not match length of index (1892563)

In [ ]:
#dfAnonimo.to_csv('anonimo.csv', sep=';', encoding='utf-8')
dfBase.head()

ANÁLISES

In [ ]:
filtro= (dfAnonimo['CPF_CNPJ_PROPRIETARIO_hs']==dfAnonimo['CPF_CONDUTOR_hs'])
dfAnonimo.loc[filtro].groupby('')    count()